In [ ]:
import numpy as np
from scipy import stats
from CTDGMR.utils import *
from CTDGMR.gmm_reduction import *
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
def gmm_plot2d(means, covs, weights, BBox, log_scale=False, X=None, file_name=None):
    x_ticks = np.linspace(BBox[0], BBox[1], 500)
    y_ticks = np.linspace(BBox[2], BBox[3], 500)
    xx, yy = np.meshgrid(x_ticks, y_ticks)
    zz = np.zeros(xx.shape)
    pos = np.empty(xx.shape + (2, ))
    pos[:, :, 0] = xx
    pos[:, :, 1] = yy
    
    fig, ax = plt.subplots(figsize = (6,6))
  
    for (l, s), w in zip(zip(means, covs), weights):
        zz += ss.multivariate_normal.pdf(pos, mean=l, cov=s) * w
    if log_scale:
        zz = np.log(zz)
    img = ax.imshow(np.flipud(zz), extent = [BBox[0], BBox[1],
                                               BBox[2], BBox[3]])   
    if X is not None:
        ax.scatter(X[:,0], X[:,1], s=1,c='r')
    if file_name is not None:
        fig.savefig(fname=file_name, dpi=200, bbox_inches = 'tight',pad_inches = 0)

# case 1

In [ ]:
eigvals = [np.array([1, 0.01])]
angles = [0, np.pi/2]

def rotate_matrix(theta):
    return np.array([np.cos(theta), -np.sin(theta), np.sin(theta), np.cos(theta)]).reshape((2,2))

b_covs = []

for i in range(len(angles)):
    rm = rotate_matrix(angles[i])
    for eigval in eigvals:
        b_covs.append(rm@np.diag(eigval)@rm.T)

b_covs = np.stack(b_covs)


b_covs = np.tile(b_covs, (4,1,1))

b_means = np.array([1,1]*2+[-1,1]*2+[-1,-1]*2+[1,-1]*2).reshape((-1,2))

K, d = b_means.shape
b_weights = np.ones((K,))/K

BBox = (-2.5,2.5,-2.5,2.5)                                                              
gmm_plot2d(b_means, b_covs, b_weights, BBox, log_scale=False, X=None)



In [ ]:
KLs = []
L2s = []
WWs = []

dists = ["KL", "WKL","W2"]
reduced_mix = {}
for dist in dists:
    obj = np.Inf
    final_rmix = None    
    for init in ["Salmond", "Runnalls", "Williams", "W", "kmeans"]:
        rmix = GMR_CTD(b_means, b_covs, b_weights, 4, n_pseudo=0, init_method=init,
                           ground_distance=dist, reg=0.1, max_iter=1000, random_state=0)
        rmix.iterative()
        if rmix.obj < obj:
            final_rmix = rmix
            obj = rmix.obj
    r_means, r_covs, r_weights = final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights
    means = [b_means, r_means]
    covs = [b_covs, r_covs]
    weights = [b_weights, r_weights]
    L2s.append(GMM_L2(means, covs, weights))
    KLs.append(MC_KL(means, covs, weights))
    WWs.append(GMM_CWD(means, covs, weights, ground_distance="W1"))
    
    reduced_mix[dist] = final_rmix
    gmm_plot2d(final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights, BBox, log_scale=False, X=None)
    
print(KLs)
print(L2s)
print(WWs)

# for WKL, increase pseduo sample size help with the final result.

# case 2

In [ ]:
def rotate_mean(r, theta):
    return np.array([r*np.cos(theta), r*np.sin(theta)])

def rotate_matrix(theta):
    return np.array([np.cos(theta), -np.sin(theta), np.sin(theta), np.cos(theta)]).reshape((2,2))

angles = np.linspace(0,2,num=16,endpoint=False) * np.pi
radius = [1, 1.5]

b_means = []
for r in radius:
    for ang in angles:
        b_means.append(rotate_mean(r, ang))

b_means = np.stack(b_means)
        
eigvals = [np.array([0.1,0.01]),np.array([0.1,0.01])]
# eigvals = [np.array([0.01,0.1]), np.array([0.01,0.1])]


b_covs = []
for eigval in eigvals:
    for i in range(angles.shape[0]):
        rm = rotate_matrix(angles[i])
        b_covs.append(rm@np.diag(eigval)@rm.T)


b_covs = np.stack(b_covs)

K, d = b_means.shape
b_weights = np.ones((K,))/K

BBox = (-2,2,-2,2)                                                              
gmm_plot2d(b_means, b_covs, b_weights, BBox, log_scale=False, X=None)



## case 3 and case 4

In [ ]:
eigvals = [np.array([0.001,1]), np.array([0.01,1]), np.array([0.1, 1])]
angles = np.linspace(0, 1, num=6, endpoint=False) * np.pi

def rotate_matrix(theta):
    return np.array([np.cos(theta), -np.sin(theta), np.sin(theta), np.cos(theta)]).reshape((2,2))

b_covs = []

for i in range(angles.shape[0]):
    rm = rotate_matrix(angles[i])
    for eigval in eigvals:
        b_covs.append(rm@np.diag(eigval)@rm.T)

b_covs = np.stack(b_covs)


K, d = len(eigvals) * angles.shape[0], 2
b_means = np.zeros((K,d))
b_weights = np.array([1/K]*K)
BBox = (-2,2,-2,2)                                                              
gmm_plot2d(b_means, b_covs, b_weights, BBox, log_scale=True, X=None)

### reduce to 6 components

In [ ]:
KLs = []
L2s = []
WWs = []

dists = ["KL", "WKL","W2"]
reduced_mix = {}
for dist in dists:
    obj = np.Inf
    final_rmix = None
    for init in ["Salmond", "Runnalls", "Williams", "W", "kmeans"]:
        print(init, dist)
        rmix = GMR_CTD(b_means, b_covs, b_weights, 6, n_pseudo=10, init_method=init,
                           ground_distance=dist, reg=0.1, max_iter=1000, random_state=0)
        rmix.iterative()
        if rmix.obj < obj:
            final_rmix = rmix
            obj = rmix.obj
    r_means, r_covs, r_weights = final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights
    means = [b_means, r_means]
    covs = [b_covs, r_covs]
    weights = [b_weights, r_weights]
    L2s.append(GMM_L2(means, covs, weights))
    KLs.append(MC_KL(means, covs, weights))
    WWs.append(GMM_CWD(means, covs, weights, ground_distance="W1"))
    
    reduced_mix[dist] = final_rmix
    gmm_plot2d(final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights, BBox, log_scale=True, X=None)
    
    
print(KLs)
print(L2s)
print(WWs)

### reduce to 12 components

In [ ]:
KLs = []
L2s = []
WWs = []

dists = ["KL", "WKL","W2"]
reduced_mix = {}
for dist in dists:
    obj = np.Inf
    final_rmix = None
    for init in ["Salmond", "Runnalls", "Williams", "W", "kmeans"]:
        rmix = GMR_CTD(b_means, b_covs, b_weights, 12, n_pseudo=1, init_method=init,
                           ground_distance=dist, reg=0.1, max_iter=1000, random_state=0)
        rmix.iterative()
        if rmix.obj < obj:
            final_rmix = rmix
            obj = rmix.obj
    r_means, r_covs, r_weights = final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights
    means = [b_means, r_means]
    covs = [b_covs, r_covs]
    weights = [b_weights, r_weights]
    L2s.append(GMM_L2(means, covs, weights))
    KLs.append(MC_KL(means, covs, weights))
    WWs.append(GMM_CWD(means, covs, weights, ground_distance="W1"))
    
    reduced_mix[dist] = final_rmix
    gmm_plot2d(final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights, BBox, log_scale=True, X=None)
    
    
print(KLs)
print(L2s)
print(WWs)

In [ ]:
KLs = []
L2s = []
WWs = []

dists = ["KL", "WKL","W2"]
reduced_mix = {}
for dist in dists:
    obj = np.Inf
    final_rmix = None
    for init in ["Salmond", "Runnalls", "Williams", "W", "kmeans"]:
        rmix = GMR_CTD(b_means, b_covs, b_weights, 16, n_pseudo=0, init_method=init,
                           ground_distance=dist, reg=1, max_iter=1000, random_state=0)
        rmix.iterative()
        
        if rmix.obj < obj:
            final_rmix = rmix
            obj = rmix.obj
    r_means, r_covs, r_weights = final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights
    means = [b_means, r_means]
    covs = [b_covs, r_covs]
    weights = [b_weights, r_weights]
    L2s.append(GMM_L2(means, covs, weights))
    KLs.append(MC_KL(means, covs, weights))
    WWs.append(GMM_CWD(means, covs, weights, ground_distance="W1"))
    
    reduced_mix[dist] = final_rmix
    gmm_plot2d(final_rmix.reduced_means, final_rmix.reduced_covs, final_rmix.reduced_weights, BBox, log_scale=False)
    
print(KLs)
print(L2s)
print(WWs)